In [2]:
from decouple import config, AutoConfig

In [3]:
config = AutoConfig(search_path='.')

In [4]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [5]:
assert config("POLYGON_API_KEY", default=None, cast=str) is not None

In [6]:
POLYGON_API_KEY = config("POLYGON_API_KEY", default=None, cast=str)

In [7]:
POLYGON_API_KEY

'XAd_OX3sZJ_1b7Cyy3D7WCpuZI2y84Oa'

In [8]:
import requests

In [9]:
ticker = "AAPL"
multiplier = "5"
timespan = "minute"
from_date = "2023-01-09"
to_date = "2023-01-09"
path = f"/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}"
url = f"https://api.polygon.io{path}?apiKey={POLYGON_API_KEY}"

In [61]:
import pytz
from typing import Literal
from datetime import datetime
from dataclasses import dataclass
from urllib.parse import urlencode

POLYGON_API_KEY = config("POLYGON_API_KEY", default=None, cast=str)

def transform_polygon_result(result):
    unix_timestamp = result.get('t') / 1000.0
    utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))
    return {
        'open_price': result['o'],
        'close_price': result['c'],
        'high_price': result['h'],
        'low_price': result['l'],
        'number_of_trades': result['n'],
        'volume': result['v'],
        'volume_weighted_average': result['vw'],
        'time': utc_timestamp,
    }

@dataclass
class PolygonAPIClient:
    ticker:str = "AAPL"
    multiplier:int = "5"
    timespan:str = "minute"
    from_date:str = "2023-01-09"
    to_date:str = "2023-01-09"
    api_key:str = ""
    adjusted:bool = True
    sort: Literal["asc", "desc"] = "asc"

    def get_params(self):
        return {
            "adjusted": self.adjusted,
            "sort": self.sort,
        }

    def get_api_key(self):
        api_key = self.api_key or POLYGON_API_KEY
        return api_key
    
    def get_headers(self):
        api_key = self.get_api_key()
        
        return {
            "Authorization": f"Bearer {api_key}",
        }
    
    def generate_url(self, pass_auth=False):
        path = f"/v2/aggs/ticker/{self.ticker}/range/{self.multiplier}/{self.timespan}/{self.from_date}/{self.to_date}"
        url = f"https://api.polygon.io{path}"
        if pass_auth:
            api_key = self.get_api_key()
            url += f"&api_key={api_key}"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        return url

    def fetch_data(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status() #not 200/201 status response
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()
        results = data['results']
        dataset = []
        for result in results:
            dataset.append(
                transform_polygon_result(result)
            )
        return dataset


In [62]:
PolygonAPIClient(ticker="GOOG", multiplier=1).generate_url(pass_auth=False)

'https://api.polygon.io/v2/aggs/ticker/GOOG/range/1/minute/2023-01-09/2023-01-09?adjusted=True&sort=asc'

In [63]:
stock_api_client = PolygonAPIClient(ticker="GOOG", multiplier=1)
dataset = stock_api_client.get_stock_data()
dataset

[{'open_price': 88.08,
  'close_price': 88.16,
  'high_price': 88.17,
  'low_price': 88.08,
  'number_of_trades': 55,
  'volume': 1684,
  'volume_weighted_average': 88.1217,
  'time': datetime.datetime(2023, 1, 9, 9, 0, tzinfo=<UTC>)},
 {'open_price': 88.37,
  'close_price': 88.37,
  'high_price': 88.37,
  'low_price': 88.37,
  'number_of_trades': 12,
  'volume': 360,
  'volume_weighted_average': 88.3648,
  'time': datetime.datetime(2023, 1, 9, 9, 2, tzinfo=<UTC>)},
 {'open_price': 88.52,
  'close_price': 88.55,
  'high_price': 88.55,
  'low_price': 88.52,
  'number_of_trades': 32,
  'volume': 1209,
  'volume_weighted_average': 88.5577,
  'time': datetime.datetime(2023, 1, 9, 9, 5, tzinfo=<UTC>)},
 {'open_price': 88.55,
  'close_price': 88.55,
  'high_price': 88.55,
  'low_price': 88.55,
  'number_of_trades': 14,
  'volume': 2169,
  'volume_weighted_average': 88.548,
  'time': datetime.datetime(2023, 1, 9, 9, 6, tzinfo=<UTC>)},
 {'open_price': 88.42,
  'close_price': 88.42,
  'high_pri